# Life Expentancy Determinator by Countries Worldwide

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
from sklearn.impute import SimpleImputer

# Preprocessing the Data for PCA
* Here we will need load, clean up, and scale the dataset
* Current cleanup steps we can determine are:
    * Remove features that do not apply to all countries
    * Focus on a subset of more recent years (maybe from 2010) forward as data is more prevalent in those years
    * Remove features that do not have hardy data
    * Determine if we should rationalize certain features where data does not exist
    * Group data by years then take the mean so that data is one value per feature

In [ ]:
# Load the worldbank_data.csv dataset. Initially we will only be using a database to process and store data.
file_path = "Resources/worldbank_data.csv"
worldbank_df = pd.read_csv(file_path)
worldbank_df

In [ ]:
worldbank_df.count()

In [ ]:
worldbank_df.rename(columns = {'Unnamed: 0':'Country'}, inplace = True)
worldbank_df.head()

In [ ]:
# Create a new DataFrame that holds only the countries names.
country_names_df = worldbank_df.filter(['Country'], axis=1)
#country_names_df = country_names_df.set_index('Symbol')
country_names_df

In [ ]:
# Drop the 'Country' column since it's not going to be used on the clustering algorithm.
worldbank_df.drop('Country', axis=1, inplace=True)
#worldbank_df = worldbank_df.set_index('Symbol')
worldbank_df

In [ ]:
# Impute NaN values using SKlearn Imputer
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer = imputer.fit(worldbank_df[['NY.GDP.MKTP.CN','NY.GDP.PCAP.CN','NY.GDP.PCAP.CD','NY.GDP.MKTP.CD','TM.UVI.MRCH.XD.WD','TX.VAL.MRCH.WL.CD','TM.VAL.MRCH.RS.ZS','TX.VAL.MRCH.XD.WD','NY.ADJ.DCO2.CD','SE.SEC.DURS','AG.LND.FRST.ZS','EN.POP.DNST','AG.LND.FRST.K2','SP.DYN.LE00.IN','SP.RUR.TOTL','TM.VAL.MRCH.XD.WD','EG.ELC.ACCS.ZS','SH.TBS.INCD','SP.DYN.LE00.FE.IN','SP.DYN.CBRT.IN','TX.UVI.MRCH.XD.WD','EG.ELC.ACCS.UR.ZS','NY.ADJ.DMIN.CD','SP.URB.TOTL','SE.PRM.AGES','TX.QTY.MRCH.XD.WD','AG.LND.TOTL.K2','PA.NUS.FCRF','SP.URB.GROW','PA.NUS.ATLS','SP.DYN.TFRT.IN','SP.POP.GROW','SP.RUR.TOTL.ZS','TM.QTY.MRCH.XD.WD','TX.VAL.MRCH.HI.ZS','TX.VAL.MRCH.RS.ZS','NY.ADJ.DNGY.CD','SE.PRM.DURS','TM.VAL.MRCH.WL.CD','NY.ADJ.AEDU.GN.ZS','SP.POP.TOTL','SP.URB.TOTL.IN.ZS','TM.VAL.MRCH.HI.ZS','TT.PRI.MRCH.XD.WD','SE.SEC.AGES','SP.DYN.CDRT.IN','SP.DYN.LE00.MA.IN','SP.RUR.TOTL.ZG'
]])
worldbank_df[['NY.GDP.MKTP.CN','NY.GDP.PCAP.CN','NY.GDP.PCAP.CD','NY.GDP.MKTP.CD','TM.UVI.MRCH.XD.WD','TX.VAL.MRCH.WL.CD','TM.VAL.MRCH.RS.ZS','TX.VAL.MRCH.XD.WD','NY.ADJ.DCO2.CD','SE.SEC.DURS','AG.LND.FRST.ZS','EN.POP.DNST','AG.LND.FRST.K2','SP.DYN.LE00.IN','SP.RUR.TOTL','TM.VAL.MRCH.XD.WD','EG.ELC.ACCS.ZS','SH.TBS.INCD','SP.DYN.LE00.FE.IN','SP.DYN.CBRT.IN','TX.UVI.MRCH.XD.WD','EG.ELC.ACCS.UR.ZS','NY.ADJ.DMIN.CD','SP.URB.TOTL','SE.PRM.AGES','TX.QTY.MRCH.XD.WD','AG.LND.TOTL.K2','PA.NUS.FCRF','SP.URB.GROW','PA.NUS.ATLS','SP.DYN.TFRT.IN','SP.POP.GROW','SP.RUR.TOTL.ZS','TM.QTY.MRCH.XD.WD','TX.VAL.MRCH.HI.ZS','TX.VAL.MRCH.RS.ZS','NY.ADJ.DNGY.CD','SE.PRM.DURS','TM.VAL.MRCH.WL.CD','NY.ADJ.AEDU.GN.ZS','SP.POP.TOTL','SP.URB.TOTL.IN.ZS','TM.VAL.MRCH.HI.ZS','TT.PRI.MRCH.XD.WD','SE.SEC.AGES','SP.DYN.CDRT.IN','SP.DYN.LE00.MA.IN','SP.RUR.TOTL.ZG'
]] = imputer.transform(worldbank_df[['NY.GDP.MKTP.CN','NY.GDP.PCAP.CN','NY.GDP.PCAP.CD','NY.GDP.MKTP.CD','TM.UVI.MRCH.XD.WD','TX.VAL.MRCH.WL.CD','TM.VAL.MRCH.RS.ZS','TX.VAL.MRCH.XD.WD','NY.ADJ.DCO2.CD','SE.SEC.DURS','AG.LND.FRST.ZS','EN.POP.DNST','AG.LND.FRST.K2','SP.DYN.LE00.IN','SP.RUR.TOTL','TM.VAL.MRCH.XD.WD','EG.ELC.ACCS.ZS','SH.TBS.INCD','SP.DYN.LE00.FE.IN','SP.DYN.CBRT.IN','TX.UVI.MRCH.XD.WD','EG.ELC.ACCS.UR.ZS','NY.ADJ.DMIN.CD','SP.URB.TOTL','SE.PRM.AGES','TX.QTY.MRCH.XD.WD','AG.LND.TOTL.K2','PA.NUS.FCRF','SP.URB.GROW','PA.NUS.ATLS','SP.DYN.TFRT.IN','SP.POP.GROW','SP.RUR.TOTL.ZS','TM.QTY.MRCH.XD.WD','TX.VAL.MRCH.HI.ZS','TX.VAL.MRCH.RS.ZS','NY.ADJ.DNGY.CD','SE.PRM.DURS','TM.VAL.MRCH.WL.CD','NY.ADJ.AEDU.GN.ZS','SP.POP.TOTL','SP.URB.TOTL.IN.ZS','TM.VAL.MRCH.HI.ZS','TT.PRI.MRCH.XD.WD','SE.SEC.AGES','SP.DYN.CDRT.IN','SP.DYN.LE00.MA.IN','SP.RUR.TOTL.ZG'
]])
worldbank_df.head()

In [ ]:
x=worldbank_df

In [ ]:
# Standardize the data with StandardScaler().
scaler = StandardScaler().fit(x)
x_scaled = scaler.transform(x)
x_scaled

# Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca = pca.fit(x_scaled)
X_pca = pca.transform(x_scaled)
print(pca.explained_variance_ratio_)

In [ ]:
# Create a DataFrame with the three principal components.  Will need to test how many PC's is best fit, could be more than 3.
pcs_df = pd.DataFrame(data=X_pca, columns=["PC1", "PC2", "PC3"])
pcs_df.head(10)

### Clustering Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1,11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


Running K-Means with `k=?`

In [ ]:
# Initialize the K-Means model. Define clusters before running
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

In [ ]:
# Concatentate the worldbank_df and pcs_df DataFrames on the same columns.
merged_df= pd.merge(worldbank_df, pcs_df, left_index=True, right_index=True)
merged_df

In [ ]:
#  Add a new column, "Country" to the clustered_df DataFrame 
merged_df["Country"] = country_names_df["Country"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
merged_df["class"] = model.labels_
pcs_df.head()

# Print the shape of the clustered_df
print(merged_df.shape)
merged_df.head(10)

### Connect to AWS RDS Database

In [ ]:
# # Configure settings for RDS
# mode = "append"
# jdbc_url="jdbc:postgresql://module16.ckjmyyscgo5g.us-east-1.rds.amazonaws.com:5432/module16db"
# config = {"user":"postgres", 
#           "password": "finalprojectgroup3", 
#           "driver":"org.postgresql.Driver"}

In [ ]:
# # Write merged_df to table in RDS
# merged_df.write.jdbc(url=jdbc_url, table='merged_data', mode=mode, properties=config)

In [1]:
import psycopg2
import config as creds


def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:

# Connecting to DB
conn, cursor = connect()

# SQL command to create inventory table
create_table = """
    CREATE TABLE IF NOT EXISTS test_table(
        index INTEGER,
        id TEXT PRIMARY KEY NOT NULL,
        category TEXT,
        image TEXT,
        displayName TEXT,
        urlHistory TEXT
    )
    """

# Execute SQL Command and commit to DB
cursor.execute(create_table)
conn.commit()

# Disconnect from DB
disconnect(conn, cursor)

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(merged_df, x='PC1', y='PC2', z='PC3',color='class', hover_name='Country', hover_data=['SP.DYN.LE00.MA.IN'])
fig.show()

In [ ]:
# Scaling data to create the scatter plot
X_cluster = merged_df[['Country', 'SP.DYN.LE00.MA.IN']].copy()
X_cluster_scaled = MinMaxScaler().fit_transform(X_cluster)
X_cluster_scaled

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(X_cluster_scaled, columns=['x', y'], index=clustered_df.index)
plot_df.head()

In [ ]:
# Create a hvplot.scatter plot using x="x" and y="y".
plot_df.hvplot.scatter(
    x="x",
    y="y",
    hover_cols=["Define"],
    by="Class",
)
